In [25]:
import os, sys
import numpy as np
import ants
import argparse
import shutil
import logging
import subprocess
import tqdm
import matplotlib.pyplot as plt 
import torchio as tio
from glob2 import glob
import json

In [53]:
import mpu.io
schw_seed = mpu.io.read('utils/bgpd_seed.json')

In [50]:
data = {'fold_0': [i[:-2] for i in schw_seed['fold_0']],
        'fold_1': [i[:-2] for i in schw_seed['fold_1']],
        'fold_2': [i[:-2] for i in schw_seed['fold_2']],
       }
print(json.dumps(data, indent=4))


# mpu.io.write('utils/bgpd_seed.json', data)

{
    "fold_0": [
        "1028_18_4",
        "1043_18_4",
        "1072_19",
        "1164_18",
        "1170_18_4",
        "1185_18_4",
        "1302_18_4",
        "1357_19_4",
        "1362_18_4",
        "1463_18_4",
        "1501_18_4",
        "1539_18",
        "1546_18",
        "1685_18_4",
        "1770_18_4",
        "349_18_4",
        "573_18_4",
        "668_18_4",
        "705_18_4",
        "746_19_4",
        "826_18_4",
        "856_19_4",
        "923_18",
        "979_18_4",
        "990_18_4",
        "Patient_1001316",
        "Patient_107017",
        "Patient_109017",
        "Patient_111016",
        "Patient_122315",
        "Patient_12417",
        "Patient_127916",
        "Patient_131416",
        "Patient_132216",
        "Patient_133916",
        "Patient_136715",
        "Patient_136915",
        "Patient_140316",
        "Patient_146716",
        "Patient_15817",
        "Patient_1815",
        "Patient_22117",
        "Patient_24717",
        "Patie

#### 1. Make tree datasets for each fold
#### 2. Train and transform images
#### 3. Check distributions for tumor, brain w/o  tumor and whole image

In [4]:
# os.mkdir('/anvar/public_datasets/preproc_study/gbm/6_hist/6_histogram/6_hist_brain_masks/')
save_dir = '/anvar/public_datasets/preproc_study/gbm/6_hist/6_histogram'
base_dir = '/anvar/public_datasets/preproc_study/gbm/3a_atlas/'

In [5]:
ants.image_read('/anvar/public_datasets/preproc_study/gbm/5_ss_shared/' + 'TCGA-02-0086/' + 'T1.nii.gz')

ANTsImage (RPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (240, 240, 155)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (-119.0, 129.0, -68.0)
	 Direction  : [ 1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [6]:
# # Saving masks in right orientation
# mask_dir = '/anvar/public_datasets/preproc_study/gbm/6_hist/6_histogram/6_hist_brain_masks/'
# orig_mask_dir = '/anvar/public_datasets/preproc_study/gbm/5_ss_shared/'
# for patient in os.listdir(base_dir):
#     if os.path.isdir(base_dir + patient):
#         mask = ants.image_read(orig_mask_dir + patient + '/CT1_mask.nii.gz')
#         mask_rpi = ants.reorient_image2(mask, orientation='LPI')
#         ants.image_write(mask_rpi, mask_dir + patient + '_mask.nii.gz', ri=False);

In [7]:
len(os.listdir(mask_dir))

102

In [8]:
# Create dataset
subjects_list = []
t1_list = []
t2_list = []
ct1_list = []
fl_list = []

for patient in os.listdir(base_dir):
    if os.path.isdir(base_dir + patient):
        subject = tio.Subject(
            t1 = tio.ScalarImage(base_dir + patient + '/T1.nii.gz'),
            t2 = tio.ScalarImage(base_dir + patient + '/T2.nii.gz'),
            ct1 = tio.ScalarImage(base_dir + patient + '/CT1.nii.gz'),
            fl = tio.ScalarImage(base_dir + patient + '/FLAIR.nii.gz'),
            label = tio.LabelMap(base_dir + patient + '/CT1_SEG.nii.gz'))

        subjects_list.append(subject)
        t1_list.append(base_dir + patient + '/T1.nii.gz')
        t2_list.append(base_dir + patient + '/T2.nii.gz')
        ct1_list.append(base_dir + patient + '/CT1.nii.gz')
        fl_list.append(base_dir + patient + '/FLAIR.nii.gz')

In [18]:
import mpu.io
gbm_seed = mpu.io.read('utils/gbm_seed.json')

In [19]:
len(gbm_seed['fold_2'])

34

In [71]:
from torchio.transforms import HistogramStandardization
import argparse
import logging
import mpu.io
from os import path


# parser = argparse.ArgumentParser()

# parser.add_argument('--path', type=str, default='/anvar/public_datasets/preproc_study/bgpd/3a_atlas/', 
#                     help='root dir for subject sequences data')
# parser.add_argument('--fixedfilename', type=list, default=['FLAIR.nii.gz'], help='name of file to register')
# parser.add_argument('--maskfilename', type=list, default=['mask_GTV_FLAIR.nii.gz'], help='name of mask to register to RPI')
# parser.add_argument('--movingfilenames', type=list, default=['CT1.nii.gz','T2.nii.gz','T1.nii.gz'], help='names of files')
# parser.add_argument('--output', type=str, default='/mnt/public_data/preproc_study/bgpd/6_hist/', 
#                     help= 'output folder')
# parser.add_argument('--seed', type=str, default='utils/example.json', help= 'mode individual or shared ')
# parser.add_argument('--device', type=str, default='cpu', help= 'gpu or cpu, if gpu - should be `int` ')

# args = parser.parse_args()
# base_dir = args.path
# save_dir = args.autput
# seed = mpu.io.read(args.seed)
# mask_name = args.maskfilename

base_dir = '/anvar/public_datasets/preproc_study/bgpd/3a_atlas/'
save_dir = '/anvar/public_datasets/preproc_study/bgpd/6_hist/'
seed = mpu.io.read('utils/bgpd_seed.json')
mask_name = 'mask_GTV_FLAIR.nii.gz'


# if __name__ == "__main__":
    
os.makedirs(save_dir, exist_ok=True)
# logging.basicConfig(filename=args.output + "logging.txt", level=logging.INFO, format='[%(asctime)s.%(msecs)03d] %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
# logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))

# Creating a dataset
subjects_list = []
for patient in os.listdir(base_dir):
    if os.path.isdir(base_dir + patient):
        subject = tio.Subject(
            t1 = tio.ScalarImage(base_dir + patient + '/T1.nii.gz'),
            t2 = tio.ScalarImage(base_dir + patient + '/T2.nii.gz'),
            ct1 = tio.ScalarImage(base_dir + patient + '/CT1.nii.gz'),
            fl = tio.ScalarImage(base_dir + patient + '/FLAIR.nii.gz'))

        subjects_list.append(subject)

# Separate dataset for each fold
for fold in ['fold_0', 'fold_1', 'fold_2']:
    print(len(seed[fold]))
#     logging.info(fold, " started.")
    # Create dataset
    temp_t1_list = []
    temp_t2_list = []
    temp_ct1_list = []
    temp_fl_list = []

    for patient in os.listdir(base_dir):
        if patient not in seed[fold]:
            if os.path.isdir(base_dir + patient):
                
                subjects_list.append(subject)
                temp_t1_list.append(base_dir + patient + '/T1.nii.gz')
                temp_t2_list.append(base_dir + patient + '/T2.nii.gz')
                temp_ct1_list.append(base_dir + patient + '/CT1.nii.gz')
                temp_fl_list.append(base_dir + patient + '/FLAIR.nii.gz')

    print('For landmarks there are ', len(temp_t1_list))
    # logging.info("Training T1 landmarks started.")
    t1_landmarks = HistogramStandardization.train(temp_t1_list)
    # logging.info("Training T2 landmarks started.")
    t2_landmarks = HistogramStandardization.train(temp_t2_list)
    # logging.info("Training CT1 landmarks started.")
    ct1_landmarks = HistogramStandardization.train(temp_ct1_list)
    # logging.info("Training FLAIR landmarks started.")
    fl_landmarks = HistogramStandardization.train(temp_fl_list)

    # Saving landmarks
    landmarks_dict = {
    't1': t1_landmarks,
    't2': t2_landmarks,
    'ct1': ct1_landmarks,
    'fl': fl_landmarks
    }

    hist_standardize = tio.HistogramStandardization(landmarks_dict)

        # Apply transforms
    for i in range(0, len(os.listdir(base_dir))):
        # Check if it is logging file instead of a folder
        if os.path.isdir(base_dir + os.listdir(base_dir)[i]):
            os.makedirs(save_dir + '/6_hist_{}/'.format(fold) + os.listdir(base_dir)[i], exist_ok= True)

            if len(os.listdir(save_dir + '/6_hist_{}/'.format(fold) + os.listdir(base_dir)[i])) < 4:
                # logging.info("Saving patient", os.listdir(base_dir)[i], fold)
                hist_standard = hist_standardize(subjects_list[i])
                hist_standard['t1'].save(save_dir + '/6_hist_{}/'.format(fold) + os.listdir(base_dir)[i] +'/T1.nii.gz')
                hist_standard['ct1'].save(save_dir + '/6_hist_{}/'.format(fold) + os.listdir(base_dir)[i] +'/CT1.nii.gz')
                hist_standard['fl'].save(save_dir + '/6_hist_{}/'.format(fold) + os.listdir(base_dir)[i] +'/FLAIR.nii.gz')
                hist_standard['t2'].save(save_dir + '/6_hist_{}/'.format(fold) + os.listdir(base_dir)[i] +'/T2.nii.gz')
                shutil.copy(base_dir + os.listdir(base_dir)[i] + '/' + mask_name,
                save_dir + '/6_hist_{}/'.format(fold) + os.listdir(base_dir)[i] +  '/' + mask_name)

  0%|          | 0/120 [00:00<?, ?it/s]

60
For landmarks there are  120


  0%|          | 0/120 [00:00<?, ?it/s]

60
For landmarks there are  120


  0%|          | 0/120 [00:00<?, ?it/s]

60
For landmarks there are  120


100%|██████████| 120/120 [01:33<00:00,  1.28it/s]


In [78]:

# parser = argparse.ArgumentParser()

# parser.add_argument('--path', type=str, default='/anvar/public_datasets/preproc_study/bgpd/3a_atlas/', 
#                     help='root dir for subject sequences data')
# parser.add_argument('--fixedfilename', type=list, default=['FLAIR.nii.gz'], help='name of file to register')
# parser.add_argument('--maskfilename', type=list, default=['mask_GTV_FLAIR.nii.gz'], help='name of mask to register to RPI')
# parser.add_argument('--movingfilenames', type=list, default=['CT1.nii.gz','T2.nii.gz','T1.nii.gz'], help='names of files')
# parser.add_argument('--output', type=str, default='/mnt/public_data/preproc_study/bgpd/6_hist/', 
#                     help= 'output folder')
# parser.add_argument('--seed', type=str, default='utils/example.json', help= 'mode individual or shared ')
# parser.add_argument('--device', type=str, default='cpu', help= 'gpu or cpu, if gpu - should be `int` ')

# args = parser.parse_args()
# base_dir = args.path
# save_dir = args.autput
# seed = mpu.io.read(args.seed)
# mask_name = args.maskfilename

base_dir = '/anvar/public_datasets/preproc_study/gbm/3a_atlas/'
save_dir = '/anvar/public_datasets/preproc_study/gbm/6_hist/6_histogram/'
seed = mpu.io.read('utils/gbm_seed.json')
mask_name = 'CT1_SEG.nii.gz'


# if __name__ == "__main__":
    
os.makedirs(save_dir, exist_ok=True)
# logging.basicConfig(filename=args.output + "logging.txt", level=logging.INFO, format='[%(asctime)s.%(msecs)03d] %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
# logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))

# Creating a dataset
subjects_list = []
for patient in os.listdir(base_dir):
    if os.path.isdir(base_dir + patient):
        subject = tio.Subject(
            t1 = tio.ScalarImage(base_dir + patient + '/T1.nii.gz'),
            t2 = tio.ScalarImage(base_dir + patient + '/T2.nii.gz'),
            ct1 = tio.ScalarImage(base_dir + patient + '/CT1.nii.gz'),
            fl = tio.ScalarImage(base_dir + patient + '/FLAIR.nii.gz'))

        subjects_list.append(subject)

# Separate dataset for each fold
for fold in ['fold_0', 'fold_1', 'fold_2']:
    print(len(seed[fold]))
#     logging.info(fold, " started.")
    # Create dataset
    temp_t1_list = []
    temp_t2_list = []
    temp_ct1_list = []
    temp_fl_list = []

    for patient in os.listdir(base_dir):
        if patient not in seed[fold]:
            if os.path.isdir(base_dir + patient):
                
                subjects_list.append(subject)
                temp_t1_list.append(base_dir + patient + '/T1.nii.gz')
                temp_t2_list.append(base_dir + patient + '/T2.nii.gz')
                temp_ct1_list.append(base_dir + patient + '/CT1.nii.gz')
                temp_fl_list.append(base_dir + patient + '/FLAIR.nii.gz')

    print('For landmarks there are ', len(temp_t1_list))
    # logging.info("Training T1 landmarks started.")
    t1_landmarks = HistogramStandardization.train(temp_t1_list)
    # logging.info("Training T2 landmarks started.")
    t2_landmarks = HistogramStandardization.train(temp_t2_list)
    # logging.info("Training CT1 landmarks started.")
    ct1_landmarks = HistogramStandardization.train(temp_ct1_list)
    # logging.info("Training FLAIR landmarks started.")
    fl_landmarks = HistogramStandardization.train(temp_fl_list)

    # Saving landmarks
    landmarks_dict = {
    't1': t1_landmarks,
    't2': t2_landmarks,
    'ct1': ct1_landmarks,
    'fl': fl_landmarks
    }

    hist_standardize = tio.HistogramStandardization(landmarks_dict)

        # Apply transforms
    for i in range(0, len(os.listdir(base_dir))):
        # Check if it is logging file instead of a folder
        if os.path.isdir(base_dir + os.listdir(base_dir)[i]):
            os.makedirs(save_dir + '/6_hist_{}/'.format(fold) + os.listdir(base_dir)[i], exist_ok= True)

            if len(os.listdir(save_dir + '/6_hist_{}/'.format(fold) + os.listdir(base_dir)[i])) < 5:
                print(os.listdir(base_dir)[i])
                # logging.info("Saving patient", os.listdir(base_dir)[i], fold)
                hist_standard = hist_standardize(subjects_list[i])
                hist_standard['t1'].save(save_dir + '/6_hist_{}/'.format(fold) + os.listdir(base_dir)[i] +'/T1.nii.gz')
                hist_standard['ct1'].save(save_dir + '/6_hist_{}/'.format(fold) + os.listdir(base_dir)[i] +'/CT1.nii.gz')
                hist_standard['fl'].save(save_dir + '/6_hist_{}/'.format(fold) + os.listdir(base_dir)[i] +'/FLAIR.nii.gz')
                hist_standard['t2'].save(save_dir + '/6_hist_{}/'.format(fold) + os.listdir(base_dir)[i] +'/T2.nii.gz')
                shutil.copy(base_dir + os.listdir(base_dir)[i] + '/' + mask_name,
                save_dir + '/6_hist_{}/'.format(fold) + os.listdir(base_dir)[i] +  '/' + mask_name)

  0%|          | 0/68 [00:00<?, ?it/s]

34
For landmarks there are  68


100%|██████████| 68/68 [00:33<00:00,  2.05it/s]


TCGA-06-0142
TCGA-02-0037


  0%|          | 0/68 [00:00<?, ?it/s]

...

TCGA-06-0142
TCGA-02-0037
TCGA-08-0520


  0%|          | 0/68 [00:00<?, ?it/s]

34
For landmarks there are  68


100%|██████████| 68/68 [00:33<00:00,  2.04it/s]


TCGA-06-0142
TCGA-02-0037
TCGA-08-0520


In [79]:
# cheching packages
for fold in ['fold_0', 'fold_1', 'fold_2']:
    print( fold, len(os.listdir(save_dir + '/6_hist_{}/'.format(fold))))
    for patient in os.listdir(base_dir):
        try:
            print(len(os.listdir(save_dir + '/6_hist_{}/'.format(fold) + patient )) == 5)
        except:
            print(patient)
            pass

fold_0 102
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
logging.txt
True
True
True
fold_1 102
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
T

## Checking distributions

In [76]:
os.listdir(save_dir)

['6_hist_fold_1', '6_hist_fold_2', '6_hist_fold_0']

In [ ]:
hist_tumor_dataframe = pd.DataFrame(columns = bins)
hist_brain_dataframe = pd.DataFrame(columns = bins)
orig_tumor_dataframe = pd.DataFrame(columns = bins)
orig_brain_dataframe = pd.DataFrame(columns = bins)

for fold in ['fold_0']:
    for patient in os.listdir(base_dir):
        if os.path.isdir(base_dir + patient):
#             print(patient)
            hist_img_path = glob(save_dir + '/6_hist_{}/'.format(fold) + patient +'/CT1.nii.gz')[0]
            orig_img_path = glob(base_dir + patient + '/CT1.nii.gz')[0]
            tumor_mask_path = glob(base_dir + patient + '/CT1_SEG.nii.gz')[0]
            brain_mask_path = glob(save_dir + '/6_hist_brain_masks/' +'{}_mask.nii.gz'.format(patient))[0]   

            hist_img = ants.image_read(hist_img_path)
            orig_img = ants.image_read(orig_img_path)
            tumor_mask = ants.image_read(tumor_mask_path)
            brain_mask = ants.image_read(brain_mask_path)

            tumor_mask = tumor_mask.numpy() > 0
            brain_mask = brain_mask.numpy() > 0
            hist_tumor= hist_img.numpy()[tumor_mask]
            orig_tumor = orig_img.numpy()[tumor_mask]

            hist_brain = hist_img.numpy()[(~tumor_mask)&brain_mask]                 
            orig_brain = orig_img.numpy()[(~tumor_mask)&brain_mask]  

            a_series = pd.Series(np.histogram(hist_tumor, bins = 100, normed =True)[0], bins )
            hist_tumor_dataframe = hist_tumor_dataframe.append(a_series, ignore_index=True)     

            b_series = pd.Series(np.histogram(hist_brain, bins = 100, normed =True)[0], bins )
            hist_brain_dataframe = hist_brain_dataframe.append(b_series, ignore_index=True)    

            c_series = pd.Series(np.histogram(orig_brain, bins = 100, normed =True)[0], bins )
            orig_brain_dataframe = orig_brain_dataframe.append(c_series, ignore_index=True)    

            d_series = pd.Series(np.histogram(orig_tumor, bins = 100, normed =True)[0], bins )
            orig_tumor_dataframe = orig_tumor_dataframe.append(d_series, ignore_index=True)      

In [ ]:
bins = np.arange(0,1, 0.01)

In [ ]:
plt.plot(hist_brain_dataframe.columns, hist_brain_dataframe.std(), label = 'Hist brain')
plt.plot(hist_brain_dataframe.columns, orig_brain_dataframe.std(), label = 'Original brain')
plt.plot(hist_brain_dataframe.columns, orig_tumor_dataframe.std(), label = 'Original tumor')
plt.plot(hist_brain_dataframe.columns, hist_tumor_dataframe.std(), label = 'Hist tumor')
plt.legend();